# 应用2 | RAG 系统：让 LLM 拥有"开卷考试"能力

---

## 🎯 本节目标

构建一个**生产级 RAG (Retrieval-Augmented Generation) 系统**：
- 使用真实的 Embedding 模型 (SentenceTransformers)
- 实现完整的向量数据库
- 连接真实 LLM 生成回答

---

## 什么是 RAG？

```
用户问题
    ↓
向量化查询 → 在知识库中搜索相似文档
    ↓
将相关文档 + 问题 发送给 LLM
    ↓
LLM 基于文档生成回答
```

**核心优势**：
- LLM 可以回答**训练数据中没有的信息**
- 减少幻觉（有文档支撑）
- 知识可以实时更新

## 运行前：准备 Ollama 后端

如果你选择 Ollama 作为后端，请先完成准备步骤：
- 参见 [PREPARE_OLLAMA.ipynb](./PREPARE_OLLAMA.ipynb)

完成后再继续本 notebook。


---

## 0. 环境配置

In [2]:
import sys
import os
sys.path.insert(0, os.path.dirname(os.getcwd()))

import numpy as np
from typing import List, Dict, Tuple, Optional
from dataclasses import dataclass
import json

# 导入统一的后端接口
from utils.llm_backend import get_llm_backend
from utils.embedding_backend import get_embedding_backend, SimpleVectorStore

print("✓ 环境准备完成!")

✓ 环境准备完成!


## 注意网络连接 使用sentence transformer

In [4]:
# ============================================================
# 配置 Embedding 和 LLM 后端
# ============================================================

# Embedding 模型 (推荐使用 sentence-transformers)
try:
    # 多语言模型，支持中英文
    embedder = get_embedding_backend(
        "sentence-transformers",
        model="paraphrase-multilingual-MiniLM-L12-v2"  # 多语言
        # model="all-MiniLM-L6-v2"  # 仅英文，更快
    )
    print(f"✓ Embedding 模型加载成功，维度: {embedder.dimension}")
except Exception as e:
    print(f"⚠️ sentence-transformers 不可用: {e}")
    print("使用 TF-IDF 备选方案...")
    from utils.embedding_backend import TFIDFEmbeddingBackend
    embedder = TFIDFEmbeddingBackend()

def check_ollama_ready(host="http://localhost:11434"):
    try:
        import requests
        resp = requests.get(f"{host}/api/tags", timeout=2)
        return resp.status_code == 200
    except Exception:
        return False

# LLM 后端
if not check_ollama_ready():
    print("Ollama 未启动。请先执行: ollama serve")
    print("并下载模型: ollama pull qwen3:4b")
try:
    llm = get_llm_backend("ollama", model="qwen3:4b")
    test = llm.chat([{"role": "user", "content": "Hi"}])
    print(f"✓ LLM 连接成功")
except Exception as e:
    print(f"⚠️ Ollama 不可用: {e}")
    try:
        llm = get_llm_backend("openai", model="gpt-3.5-turbo")
        print("✓ 使用 OpenAI 后端")
    except:
        llm = None
        print("❌ 请配置 LLM 后端")

正在加载 Embedding 模型: paraphrase-multilingual-MiniLM-L12-v2...
✓ 模型加载完成! 维度: 384
✓ Embedding 模型加载成功，维度: 384
✓ LLM 连接成功


---

## 1. 向量数据库实现

我们使用 `SimpleVectorStore`，它支持：
- 添加文档并自动向量化
- 余弦相似度搜索
- 持久化存储

In [5]:
# 创建向量数据库
vector_store = SimpleVectorStore(embedder)

# 准备知识库文档
documents = [
    # LLM 基础知识
    "Transformer是2017年Google在论文'Attention is All You Need'中提出的深度学习架构。它引入了自注意力机制，能够并行处理序列数据，成为现代大语言模型的基础。",
    "GPT (Generative Pre-trained Transformer) 是OpenAI开发的语言模型系列。GPT-3有1750亿参数，GPT-4是多模态模型，能够处理图像和文本。",
    "BERT (Bidirectional Encoder Representations from Transformers) 是Google开发的预训练语言模型，采用双向Transformer编码器，在NLU任务上表现优秀。",
    "LLaMA是Meta开发的开源大语言模型系列。LLaMA 2有7B、13B、70B三个版本，支持商业使用。LLaMA 3进一步提升了性能。",
    "ChatGPT是基于GPT-3.5和GPT-4的对话AI，于2022年11月发布，两个月内用户突破1亿。它使用RLHF技术进行对齐训练。",
    
    # 训练技术
    "预训练(Pre-training)是在大规模无标注文本上训练语言模型，让模型学习语言的统计规律。常用目标包括Next Token Prediction和Masked Language Modeling。",
    "SFT (Supervised Fine-Tuning) 是监督微调，使用高质量的指令-回答对训练模型，让模型学会遵循指令。这是把Base Model变成Chat Model的关键步骤。",
    "RLHF (Reinforcement Learning from Human Feedback) 使用人类反馈来优化模型。流程包括：1)训练Reward Model 2)使用PPO优化策略模型。",
    "DPO (Direct Preference Optimization) 是一种简化的对齐方法，直接从偏好数据学习，不需要训练单独的Reward Model，训练更稳定。",
    "LoRA (Low-Rank Adaptation) 是一种参数高效微调方法，只训练低秩矩阵，大大减少可训练参数量，让消费级显卡也能微调大模型。",
    
    # 技术细节
    "自注意力机制(Self-Attention)让模型能够关注输入序列中的不同位置，通过Q、K、V矩阵计算注意力权重，捕捉长距离依赖关系。",
    "KV Cache是推理加速技术，缓存已计算的Key和Value矩阵，避免重复计算，可以显著提升自回归生成的速度。",
    "量化(Quantization)通过降低参数精度(FP16→INT8→INT4)来减少内存占用。4-bit量化可以让7B模型只需要4GB显存。",
    "RAG (Retrieval-Augmented Generation) 结合检索和生成，先从知识库检索相关文档，再让LLM基于文档生成回答，减少幻觉。",
    "Agent是能够自主规划和执行任务的AI系统。ReAct模式让Agent通过思考-行动-观察循环来解决问题，可以调用外部工具。",
]

# 添加元数据
metadata = [
    {"topic": "architecture", "id": 1},
    {"topic": "models", "id": 2},
    {"topic": "models", "id": 3},
    {"topic": "models", "id": 4},
    {"topic": "models", "id": 5},
    {"topic": "training", "id": 6},
    {"topic": "training", "id": 7},
    {"topic": "training", "id": 8},
    {"topic": "training", "id": 9},
    {"topic": "training", "id": 10},
    {"topic": "technical", "id": 11},
    {"topic": "technical", "id": 12},
    {"topic": "technical", "id": 13},
    {"topic": "technical", "id": 14},
    {"topic": "technical", "id": 15},
]

# 添加到向量数据库
vector_store.add_documents(documents, metadata)

✓ 已添加 15 个文档，总计 15 个


---

## 2. 测试向量搜索

In [6]:
def pretty_search(query: str, top_k: int = 3):
    """格式化显示搜索结果"""
    print(f"\n🔍 查询: {query}")
    print("-" * 60)
    
    results = vector_store.search(query, top_k=top_k)
    
    for i, result in enumerate(results, 1):
        print(f"\n📄 结果 {i} (相似度: {result['score']:.4f})")
        print(f"   主题: {result['metadata'].get('topic', 'N/A')}")
        print(f"   内容: {result['document'][:100]}...")
    
    return results

# 测试搜索
pretty_search("什么是Transformer？")


🔍 查询: 什么是Transformer？
------------------------------------------------------------

📄 结果 1 (相似度: 0.5470)
   主题: architecture
   内容: Transformer是2017年Google在论文'Attention is All You Need'中提出的深度学习架构。它引入了自注意力机制，能够并行处理序列数据，成为现代大语言模型的基础。...

📄 结果 2 (相似度: 0.5257)
   主题: models
   内容: BERT (Bidirectional Encoder Representations from Transformers) 是Google开发的预训练语言模型，采用双向Transformer编码器，...

📄 结果 3 (相似度: 0.3722)
   主题: models
   内容: GPT (Generative Pre-trained Transformer) 是OpenAI开发的语言模型系列。GPT-3有1750亿参数，GPT-4是多模态模型，能够处理图像和文本。...


[{'document': "Transformer是2017年Google在论文'Attention is All You Need'中提出的深度学习架构。它引入了自注意力机制，能够并行处理序列数据，成为现代大语言模型的基础。",
  'score': 0.547032356262207,
  'metadata': {'topic': 'architecture', 'id': 1}},
 {'document': 'BERT (Bidirectional Encoder Representations from Transformers) 是Google开发的预训练语言模型，采用双向Transformer编码器，在NLU任务上表现优秀。',
  'score': 0.5257304310798645,
  'metadata': {'topic': 'models', 'id': 3}},
 {'document': 'GPT (Generative Pre-trained Transformer) 是OpenAI开发的语言模型系列。GPT-3有1750亿参数，GPT-4是多模态模型，能够处理图像和文本。',
  'score': 0.3721601963043213,
  'metadata': {'topic': 'models', 'id': 2}}]

In [7]:
# 更多搜索测试
pretty_search("如何训练大语言模型？")


🔍 查询: 如何训练大语言模型？
------------------------------------------------------------

📄 结果 1 (相似度: 0.6368)
   主题: training
   内容: 预训练(Pre-training)是在大规模无标注文本上训练语言模型，让模型学习语言的统计规律。常用目标包括Next Token Prediction和Masked Language Modeling...

📄 结果 2 (相似度: 0.4671)
   主题: training
   内容: SFT (Supervised Fine-Tuning) 是监督微调，使用高质量的指令-回答对训练模型，让模型学会遵循指令。这是把Base Model变成Chat Model的关键步骤。...

📄 结果 3 (相似度: 0.4565)
   主题: models
   内容: BERT (Bidirectional Encoder Representations from Transformers) 是Google开发的预训练语言模型，采用双向Transformer编码器，...


[{'document': '预训练(Pre-training)是在大规模无标注文本上训练语言模型，让模型学习语言的统计规律。常用目标包括Next Token Prediction和Masked Language Modeling。',
  'score': 0.6367638111114502,
  'metadata': {'topic': 'training', 'id': 6}},
 {'document': 'SFT (Supervised Fine-Tuning) 是监督微调，使用高质量的指令-回答对训练模型，让模型学会遵循指令。这是把Base Model变成Chat Model的关键步骤。',
  'score': 0.46705737709999084,
  'metadata': {'topic': 'training', 'id': 7}},
 {'document': 'BERT (Bidirectional Encoder Representations from Transformers) 是Google开发的预训练语言模型，采用双向Transformer编码器，在NLU任务上表现优秀。',
  'score': 0.45646658539772034,
  'metadata': {'topic': 'models', 'id': 3}}]

In [8]:
pretty_search("LoRA 是什么？如何减少显存？")


🔍 查询: LoRA 是什么？如何减少显存？
------------------------------------------------------------

📄 结果 1 (相似度: 0.6748)
   主题: training
   内容: LoRA (Low-Rank Adaptation) 是一种参数高效微调方法，只训练低秩矩阵，大大减少可训练参数量，让消费级显卡也能微调大模型。...

📄 结果 2 (相似度: 0.3575)
   主题: models
   内容: LLaMA是Meta开发的开源大语言模型系列。LLaMA 2有7B、13B、70B三个版本，支持商业使用。LLaMA 3进一步提升了性能。...

📄 结果 3 (相似度: 0.3319)
   主题: technical
   内容: RAG (Retrieval-Augmented Generation) 结合检索和生成，先从知识库检索相关文档，再让LLM基于文档生成回答，减少幻觉。...


[{'document': 'LoRA (Low-Rank Adaptation) 是一种参数高效微调方法，只训练低秩矩阵，大大减少可训练参数量，让消费级显卡也能微调大模型。',
  'score': 0.6748276948928833,
  'metadata': {'topic': 'training', 'id': 10}},
 {'document': 'LLaMA是Meta开发的开源大语言模型系列。LLaMA 2有7B、13B、70B三个版本，支持商业使用。LLaMA 3进一步提升了性能。',
  'score': 0.35746684670448303,
  'metadata': {'topic': 'models', 'id': 4}},
 {'document': 'RAG (Retrieval-Augmented Generation) 结合检索和生成，先从知识库检索相关文档，再让LLM基于文档生成回答，减少幻觉。',
  'score': 0.33194488286972046,
  'metadata': {'topic': 'technical', 'id': 14}}]

In [9]:
# 英文查询也可以
pretty_search("What is RLHF and how does it work?")


🔍 查询: What is RLHF and how does it work?
------------------------------------------------------------

📄 结果 1 (相似度: 0.4759)
   主题: training
   内容: RLHF (Reinforcement Learning from Human Feedback) 使用人类反馈来优化模型。流程包括：1)训练Reward Model 2)使用PPO优化策略模型。...

📄 结果 2 (相似度: 0.3984)
   主题: models
   内容: LLaMA是Meta开发的开源大语言模型系列。LLaMA 2有7B、13B、70B三个版本，支持商业使用。LLaMA 3进一步提升了性能。...

📄 结果 3 (相似度: 0.3848)
   主题: models
   内容: ChatGPT是基于GPT-3.5和GPT-4的对话AI，于2022年11月发布，两个月内用户突破1亿。它使用RLHF技术进行对齐训练。...


[{'document': 'RLHF (Reinforcement Learning from Human Feedback) 使用人类反馈来优化模型。流程包括：1)训练Reward Model 2)使用PPO优化策略模型。',
  'score': 0.47592031955718994,
  'metadata': {'topic': 'training', 'id': 8}},
 {'document': 'LLaMA是Meta开发的开源大语言模型系列。LLaMA 2有7B、13B、70B三个版本，支持商业使用。LLaMA 3进一步提升了性能。',
  'score': 0.39844903349876404,
  'metadata': {'topic': 'models', 'id': 4}},
 {'document': 'ChatGPT是基于GPT-3.5和GPT-4的对话AI，于2022年11月发布，两个月内用户突破1亿。它使用RLHF技术进行对齐训练。',
  'score': 0.38481268286705017,
  'metadata': {'topic': 'models', 'id': 5}}]

---

## 3. 构建完整 RAG 系统

In [10]:
class RAGSystem:
    """
    完整的 RAG 系统
    
    组件:
    - Embedding Model: 文本向量化
    - Vector Store: 存储和检索向量
    - LLM: 生成回答
    """
    
    SYSTEM_PROMPT = """你是一个知识问答助手。请基于提供的参考资料回答用户问题。

规则:
1. 只使用参考资料中的信息回答
2. 如果参考资料中没有相关信息，请明确说明
3. 引用具体内容时请准确
4. 回答要简洁准确

参考资料:
{context}
"""
    
    def __init__(self, vector_store: SimpleVectorStore, llm):
        self.vector_store = vector_store
        self.llm = llm
    
    def retrieve(self, query: str, top_k: int = 3) -> List[Dict]:
        """检索相关文档"""
        return self.vector_store.search(query, top_k=top_k)
    
    def format_context(self, results: List[Dict]) -> str:
        """格式化检索结果为上下文"""
        context_parts = []
        for i, result in enumerate(results, 1):
            context_parts.append(f"[文档{i}] {result['document']}")
        return "\n\n".join(context_parts)
    
    def answer(self, question: str, top_k: int = 3, verbose: bool = True) -> str:
        """
        RAG 完整流程：检索 + 生成
        
        Args:
            question: 用户问题
            top_k: 检索文档数量
            verbose: 是否显示详细过程
            
        Returns:
            str: 生成的回答
        """
        if verbose:
            print(f"\n{'='*60}")
            print(f"🤔 问题: {question}")
            print(f"{'='*60}")
        
        # Step 1: 检索
        if verbose:
            print(f"\n📚 Step 1: 检索相关文档...")
        
        results = self.retrieve(question, top_k)
        
        if verbose:
            print(f"找到 {len(results)} 个相关文档:")
            for i, r in enumerate(results, 1):
                print(f"  {i}. [{r['score']:.3f}] {r['document'][:50]}...")
        
        if not results:
            return "抱歉，没有找到相关信息。"
        
        # Step 2: 构建 Prompt
        context = self.format_context(results)
        system_prompt = self.SYSTEM_PROMPT.format(context=context)
        
        if verbose:
            print(f"\n📝 Step 2: 构建 Prompt...")
            print(f"上下文长度: {len(context)} 字符")
        
        # Step 3: LLM 生成回答
        if verbose:
            print(f"\n🤖 Step 3: LLM 生成回答...")
        
        if self.llm is None:
            # 没有 LLM，返回检索结果摘要
            answer = f"[检索结果摘要]\n基于检索到的文档:\n{context}"
        else:
            messages = [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": question}
            ]
            answer = self.llm.chat(messages, temperature=0.3)
        
        if verbose:
            print(f"\n✅ 回答:\n{answer}")
        
        return answer
    
    def answer_with_sources(self, question: str, top_k: int = 3) -> Dict:
        """
        带来源的回答
        
        Returns:
            dict: {"answer": str, "sources": List[Dict]}
        """
        results = self.retrieve(question, top_k)
        
        if not results:
            return {
                "answer": "抱歉，没有找到相关信息。",
                "sources": []
            }
        
        context = self.format_context(results)
        system_prompt = self.SYSTEM_PROMPT.format(context=context)
        
        if self.llm:
            messages = [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": question}
            ]
            answer = self.llm.chat(messages, temperature=0.3)
        else:
            answer = results[0]["document"]
        
        return {
            "answer": answer,
            "sources": [
                {
                    "content": r["document"],
                    "score": r["score"],
                    "metadata": r["metadata"]
                }
                for r in results
            ]
        }

# 创建 RAG 系统
rag = RAGSystem(vector_store, llm)
print("\n✓ RAG 系统创建成功!")


✓ RAG 系统创建成功!


---

## 4. 测试 RAG 系统

In [11]:
# 测试 1: 基础问答
rag.answer("什么是 Transformer？它有什么特点？")


🤔 问题: 什么是 Transformer？它有什么特点？

📚 Step 1: 检索相关文档...
找到 3 个相关文档:
  1. [0.558] Transformer是2017年Google在论文'Attention is All You Ne...
  2. [0.526] BERT (Bidirectional Encoder Representations from T...
  3. [0.377] GPT (Generative Pre-trained Transformer) 是OpenAI开发...

📝 Step 2: 构建 Prompt...
上下文长度: 327 字符

🤖 Step 3: LLM 生成回答...

✅ 回答:
Transformer是2017年Google在论文'Attention is All You Need'中提出的深度学习架构。它引入了自注意力机制，能够并行处理序列数据，成为现代大语言模型的基础。


"Transformer是2017年Google在论文'Attention is All You Need'中提出的深度学习架构。它引入了自注意力机制，能够并行处理序列数据，成为现代大语言模型的基础。"

In [12]:
# 测试 2: 训练相关
rag.answer("如何让 Base Model 变成 Chat Model？")


🤔 问题: 如何让 Base Model 变成 Chat Model？

📚 Step 1: 检索相关文档...
找到 3 个相关文档:
  1. [0.426] SFT (Supervised Fine-Tuning) 是监督微调，使用高质量的指令-回答对训练模...
  2. [0.424] ChatGPT是基于GPT-3.5和GPT-4的对话AI，于2022年11月发布，两个月内用户突破1...
  3. [0.400] 自注意力机制(Self-Attention)让模型能够关注输入序列中的不同位置，通过Q、K、V矩阵计...

📝 Step 2: 构建 Prompt...
上下文长度: 250 字符

🤖 Step 3: LLM 生成回答...

✅ 回答:
根据参考资料，SFT (Supervised Fine-Tuning) 是把Base Model变成Chat Model的关键步骤。它使用高质量的指令-回答对训练模型，让模型学会遵循指令。


'根据参考资料，SFT (Supervised Fine-Tuning) 是把Base Model变成Chat Model的关键步骤。它使用高质量的指令-回答对训练模型，让模型学会遵循指令。'

In [13]:
# 测试 3: 技术细节
rag.answer("LoRA 和量化如何帮助降低显存？")


🤔 问题: LoRA 和量化如何帮助降低显存？

📚 Step 1: 检索相关文档...
找到 3 个相关文档:
  1. [0.733] LoRA (Low-Rank Adaptation) 是一种参数高效微调方法，只训练低秩矩阵，大大减...
  2. [0.534] 量化(Quantization)通过降低参数精度(FP16→INT8→INT4)来减少内存占用。4-...
  3. [0.385] 自注意力机制(Self-Attention)让模型能够关注输入序列中的不同位置，通过Q、K、V矩阵计...

📝 Step 2: 构建 Prompt...
上下文长度: 232 字符

🤖 Step 3: LLM 生成回答...

✅ 回答:
根据参考资料：

- **LoRA** 通过只训练低秩矩阵，大大减少可训练参数量，从而降低显存需求（[文档1]）。
- **量化** 通过降低参数精度（如 FP16→INT8→INT4）来减少内存占用，例如 4-bit 量化可以让 7B 模型只需 4GB 显存（[文档2]）。

以上机制均直接针对显存优化，符合参考资料描述。


'根据参考资料：\n\n- **LoRA** 通过只训练低秩矩阵，大大减少可训练参数量，从而降低显存需求（[文档1]）。\n- **量化** 通过降低参数精度（如 FP16→INT8→INT4）来减少内存占用，例如 4-bit 量化可以让 7B 模型只需 4GB 显存（[文档2]）。\n\n以上机制均直接针对显存优化，符合参考资料描述。'

In [14]:
# 测试 4: 带来源的回答
result = rag.answer_with_sources("RLHF 和 DPO 有什么区别？")
print(f"\n📝 回答: {result['answer']}")
print(f"\n📚 来源:")
for i, source in enumerate(result['sources'], 1):
    print(f"  {i}. [{source['score']:.3f}] {source['content'][:80]}...")


📝 回答: 根据参考资料：  
- **RLHF**（Reinforcement Learning from Human Feedback）的流程包括：先训练Reward Model，再使用PPO优化策略模型。  
- **DPO**（Direct Preference Optimization）直接从偏好数据学习，不需要训练单独的Reward Model，训练更稳定。  

**主要区别**：RLHF需要额外训练Reward Model和PPO优化步骤，而DPO省去了训练Reward Model的环节，简化了流程并提升了训练稳定性。

📚 来源:
  1. [0.479] DPO (Direct Preference Optimization) 是一种简化的对齐方法，直接从偏好数据学习，不需要训练单独的Reward Model，训...
  2. [0.413] RLHF (Reinforcement Learning from Human Feedback) 使用人类反馈来优化模型。流程包括：1)训练Reward Mo...
  3. [0.381] LLaMA是Meta开发的开源大语言模型系列。LLaMA 2有7B、13B、70B三个版本，支持商业使用。LLaMA 3进一步提升了性能。...


In [15]:
# 测试 5: 知识库中不存在的问题
rag.answer("Python 的 GIL 是什么？")


🤔 问题: Python 的 GIL 是什么？

📚 Step 1: 检索相关文档...
找到 3 个相关文档:
  1. [0.358] GPT (Generative Pre-trained Transformer) 是OpenAI开发...
  2. [0.316] ChatGPT是基于GPT-3.5和GPT-4的对话AI，于2022年11月发布，两个月内用户突破1...
  3. [0.248] KV Cache是推理加速技术，缓存已计算的Key和Value矩阵，避免重复计算，可以显著提升自回归...

📝 Step 2: 构建 Prompt...
上下文长度: 240 字符

🤖 Step 3: LLM 生成回答...

✅ 回答:
参考资料中没有提供Python GIL的相关信息。提供的参考资料仅涉及GPT模型系列、ChatGPT和KV Cache等AI技术内容，与Python编程中的GIL（Global Interpreter Lock）无关。


'参考资料中没有提供Python GIL的相关信息。提供的参考资料仅涉及GPT模型系列、ChatGPT和KV Cache等AI技术内容，与Python编程中的GIL（Global Interpreter Lock）无关。'

---

## 5. 高级功能：文档分块与重排序

In [16]:
class DocumentChunker:
    """
    文档分块器
    
    长文档需要分成小块才能有效检索
    """
    
    def __init__(self, chunk_size: int = 500, chunk_overlap: int = 50):
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap
    
    def chunk_text(self, text: str, metadata: Dict = None) -> List[Dict]:
        """
        将文本分成重叠的小块
        
        Args:
            text: 原始文本
            metadata: 元数据
            
        Returns:
            List[Dict]: [{"text": str, "metadata": dict}, ...]
        """
        if metadata is None:
            metadata = {}
        
        chunks = []
        start = 0
        chunk_id = 0
        
        while start < len(text):
            end = start + self.chunk_size
            
            # 尝试在句子边界分割
            if end < len(text):
                # 寻找最近的句号
                for punct in ['。', '.', '！', '!', '？', '?', '\n']:
                    last_punct = text[start:end].rfind(punct)
                    if last_punct > self.chunk_size // 2:
                        end = start + last_punct + 1
                        break
            
            chunk_text = text[start:end].strip()
            if chunk_text:
                chunk_metadata = {
                    **metadata,
                    "chunk_id": chunk_id,
                    "start": start,
                    "end": end
                }
                chunks.append({
                    "text": chunk_text,
                    "metadata": chunk_metadata
                })
                chunk_id += 1
            
            start = end - self.chunk_overlap
        
        return chunks
    
    def chunk_documents(self, documents: List[str], metadatas: List[Dict] = None) -> Tuple[List[str], List[Dict]]:
        """
        批量分块文档
        
        Returns:
            (texts, metadatas)
        """
        if metadatas is None:
            metadatas = [{} for _ in documents]
        
        all_texts = []
        all_metadata = []
        
        for doc, meta in zip(documents, metadatas):
            chunks = self.chunk_text(doc, meta)
            for chunk in chunks:
                all_texts.append(chunk["text"])
                all_metadata.append(chunk["metadata"])
        
        return all_texts, all_metadata

# 测试分块
chunker = DocumentChunker(chunk_size=200, chunk_overlap=30)

long_doc = """
大语言模型(Large Language Model, LLM)是指参数量巨大的语言模型。
这些模型通常基于Transformer架构，通过在海量文本数据上进行预训练来学习语言的统计规律。
预训练完成后，模型可以通过微调(Fine-tuning)来适应特定任务。
常见的微调方法包括SFT(监督微调)、RLHF(基于人类反馈的强化学习)和DPO(直接偏好优化)。
为了降低微调的计算成本，研究人员提出了LoRA等参数高效微调方法。
在推理阶段，KV Cache技术可以加速自回归生成，量化技术可以减少内存占用。
"""

chunks = chunker.chunk_text(long_doc, {"source": "intro"})
print(f"原文长度: {len(long_doc)} 字符")
print(f"分成 {len(chunks)} 个块:")
for i, chunk in enumerate(chunks):
    print(f"\n块 {i+1} ({len(chunk['text'])} 字符):")
    print(f"  {chunk['text'][:100]}...")

原文长度: 257 字符
分成 2 个块:

块 1 (181 字符):
  大语言模型(Large Language Model, LLM)是指参数量巨大的语言模型。
这些模型通常基于Transformer架构，通过在海量文本数据上进行预训练来学习语言的统计规律。
预训练完成...

块 2 (104 字符):
  RLHF(基于人类反馈的强化学习)和DPO(直接偏好优化)。
为了降低微调的计算成本，研究人员提出了LoRA等参数高效微调方法。
在推理阶段，KV Cache技术可以加速自回归生成，量化技术可以减少内...


---

## 本节总结


1. **真实的 Embedding 模型**
   - SentenceTransformers 生成语义向量
   - 支持多语言检索

2. **向量数据库**
   - 余弦相似度搜索
   - 元数据过滤
   - 持久化存储

3. **RAG 完整流程**
   - 检索 → 重排 → 生成
   - 带来源的回答

### 扩展练习

1. **添加 PDF 解析**: 使用 PyPDF2 加载 PDF 文档
2. **接入 ChromaDB**: 使用专业的向量数据库
3. **实现混合检索**: 结合关键词和语义搜索

In [19]:
# 练习: 添加从文件加载文档的功能

def load_text_file(file_path: str) -> str:
    """加载文本文件"""
    with open(file_path, 'r', encoding='utf-8') as f:
        return f.read()

def load_markdown_file(file_path: str) -> List[str]:
    """加载 Markdown 文件，按标题分割"""
    import re
    content = load_text_file(file_path)
    
    # 按 ## 标题分割
    sections = re.split(r'\n## ', content)
    return [s.strip() for s in sections if s.strip()]

# 示例用法 (需要实际文件)
# docs = load_markdown_file('knowledge_base.md')
# vector_store.add_documents(docs)

#print("✓ 文档加载器已定义")